In [12]:
import requests
import pandas as pd
import os
import json

camara_base_url = 'https://dadosabertos.camara.leg.br/api/v2/'

url = f'{camara_base_url}/deputados'
response = requests.get(url)
if not response.ok:
    raise Exception('Nao foi possivel recuperar os dados')

df_deputados = pd.DataFrame().from_dict(json.loads(response.text)['dados'])

df_deputados[['nome', 'siglaPartido']].to_parquet('data/deputados.parquet')

In [13]:
df_parquet = pd.read_parquet('data/deputados.parquet')
df_parquet.head()

,nome,siglaPartido
0,Abilio Brunini,PL
1,Acácio Favacho,MDB
2,Adail Filho,REPUBLICANOS
3,Adilson Barroso,PL
4,Adolfo Viana,PSDB


In [15]:
print(df_parquet.to_string())

                                    nome   siglaPartido
0                         Abilio Brunini             PL
1                         Acácio Favacho            MDB
2                            Adail Filho   REPUBLICANOS
3                        Adilson Barroso             PL
4                           Adolfo Viana           PSDB
5                        Adriana Ventura           NOVO
6                       Adriano do Baldy             PP
7                            Aécio Neves           PSDB
8                            Afonso Hamm             PP
9                           Afonso Motta            PDT
10                     Aguinaldo Ribeiro             PP
11                        Airton Faleiro             PT
12                        AJ Albuquerque             PP
13                         Alberto Fraga             PL
14                        Alberto Mourão            MDB
15                           Albuquerque   REPUBLICANOS
16                         Alceu Moreira        